In [1]:
import gym
import numpy as np
import tensorflow as tf
import random
import time

from IPython.display import clear_output
import time

In [2]:
print(tf.__version__)

1.12.0


In [5]:
env = gym.make('FrozenLake-v0')
env.reset()

0

In [6]:
# Creating a neural network

tf.reset_default_graph()

with tf.variable_scope("model"):
    inputs = tf.placeholder(name="inputs", shape=[None, env.observation_space.n], dtype=tf.float32)
    W = tf.get_variable(name="W", shape=[env.observation_space.n, env.action_space.n], dtype=tf.float32,
                       initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="b", initializer=tf.zeros(shape=[env.action_space.n]), dtype=tf.float32)
    
q_pred = tf.add(tf.matmul(inputs, W), b)
a_pred = tf.argmax(q_pred, axis=1)

q_target = tf.placeholder(shape=[1, env.action_space.n], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(q_target - q_pred))

train = tf.train.AdamOptimizer(learning_rate=0.001)
minimizer = train.minimize(loss)

In [9]:
# Training the neural network

init = tf.global_variables_initializer()

discount = 0.9
epsilon = 0.1
episodes = 1000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(episodes):
        s = env.reset()
        total_rewards = 0
        
        while True:
            pred_a, pred_q = sess.run([a_pred, q_pred], 
                                      feed_dict={inputs: np.identity(env.observation_space.n)[s: s+1]})
            
            if np.random.uniform(0, 1) < epsilon:
                pred_a[0] = env.action_space.sample()
                
            new_state, reward, done, info = env.step(pred_a[0])

            if reward == 0:
                if done == True:
                    reward = -5
                else:
                    reward = -1

            if reward:
                reward = 10
            
            pred_q_new = sess.run(q_pred, 
                                feed_dict={inputs:np.identity(env.observation_space.n)[new_state:new_state+1]})
            
            target_q = pred_q
            max_q_pred = np.max(pred_q_new)
            target_q[0, pred_a[0]] = reward + discount * max_q_pred
            
            sess.run(minimizer, feed_dict={
                inputs: np.identity(env.observation_space.n)[s: s+1], q_target: target_q})
            s = new_state
            
            if done:
                break
    
    tf.summary.FileWriter("logs/q_network", sess.graph)
    print("Output after learning")
    s = env.reset()
    env.render()
    
    while True:
        a = sess.run(a_pred, feed_dict={inputs: np.identity(env.observation_space.n)[s: s+1]})
        new_state, reward, done, info = env.step(a[0])
        print("=" * 20)
        env.render()
        s = new_state
        
        if done:
            break

Output after learning

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
